In [ ]:
import numpy as np
import pandas as pd

In [ ]:
base_path = '../input/'

# Model Toxic Comment Classification Challenge

In [ ]:
jf_train_df = pd.read_csv(base_path + 'jigsaw-toxic-comment-classification-challenge/train.csv')

In [ ]:
# 가중치 부여 가공
toxic = 1.0
severe_toxic = 2.0
obscene = 1.0
threat = 1.0
insult = 1.0
identity_hate = 2.0

def create_train_first(df):
    df['y'] = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
    df['y'] = df['y'] + df['severe_toxic']*severe_toxic
    df['y'] = df['y'] + df['obscene']*obscene
    df['y'] = df['y'] + df['threat']*threat
    df['y'] = df['y'] + df['insult']*insult
    df['y'] = df['y'] + df['identity_hate']*identity_hate
    
    # renameing은 원하는대로 변경 가능
#     df= df[['comment_text', 'y', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].rename(columns={'comment_text': 'text'})
    
    #undersample
    min_len = (df['y'] >= 1).sum()
    df_y0_undersample = df[df['y'] == 0].sample(n=int(min_len*1.5), random_state=201)
    df = pd.concat([df[df['y'] >= 1], df_y0_undersample])
    
    return df


In [ ]:
jf_train_df = create_train_first(jf_train_df)
print(jf_train_df['y'].value_counts())

# Model Unintended Bias in Toxicity Classification

In [ ]:
js_train_df = pd.read_csv(base_path + 'jigsaw-unintended-bias-in-toxicity-classification/train.csv')

In [ ]:
def create_train_second(df):
    # non-toxic data remove by annotator_count
    df = df.query('toxicity_annotator_count > 5')
    
    df['y'] = df[['severe_toxicity', 'obscene', 'sexual_explicit', 'identity_attack', 'insult', 'threat']].sum(axis=1)
    df['y'] = df.apply(lambda row: row['target'] if row['target'] <= 0.5 else row['y'], axis=1)
    
    # renameing은 원하는대로 변경 가능
#     js_train_df = js_train_df[['comment_text', 'y']].rename(columns={'comment_text': 'text'})

    # undersampling
    min_len = (df['y'] > 0.5).sum()
    df_y0_undersample = df[df['y'] <= 0.5].sample(n=int(min_len*1.5), random_state=201)
    df = pd.concat([df[df['y'] > 0.5], df_y0_undersample])
    
    return df

In [ ]:
js_train_df = create_train_second(js_train_df)
print(js_train_df['y'].value_counts())

# Model Ruddit: Norms of Offensiveness for English Reddit Comments

In [ ]:
rud_df = pd.read_csv(base_path + 'ruddit-jigsaw-dataset/Dataset/ruddit_with_text.csv')

In [ ]:
rud_df['y'] = rud_df['offensiveness_score'].map(lambda x: 0.0 if x <=0 else x)
# rud_df = rud_df[['txt', 'y']].rename(columns={'txt': 'text'})
print(rud_df['y'].value_counts())